In [4]:
# Google drive로 Mount

from google.colab import drive
drive.mount('/content/drive/')

# 폴더 이동
%cd '/content/drive/My Drive/hanhwa_AA/Hanhwa_AA/2_Load_forecast/ME-DeepL/'
 #수정 필요

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/hanhwa_AA/Hanhwa_AA/2_Load_forecast/ME-DeepL


In [6]:
# -*- coding: utf-8 -*-
"""Proposed_hybrid multimodal_model_COD.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1cRewIg4RXvfeY1nRS34W0pj73MnTtXLG
"""

# Commented out IPython magic to ensure Python compatibility.
import os

#s.environ["CUDA_VISIBLE_DEVICES"] = '0'  # ,1,2,3,4,5'

# call libraries
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras import layers
from tensorflow.keras.layers import GRU, ReLU, LeakyReLU, ELU
from keras.layers import Dropout
from tensorflow.keras.layers import Conv1D, Conv2D, ConvLSTM2D, MaxPool1D
from tensorflow.keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import math
import pandas as pd
import scipy.io as sio
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import time


def mean_absolute_percentage_error(y_true, y_pred):
    ## Note: does not handle mix 1d representation
    # if _is_1d(y_true):
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


from keras.layers import CuDNNGRU, CuDNNLSTM, Activation

# from tensorflow.keras.utils import multi_gpu_model


print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")


# tf.compat.v1.disable_eager_execution()

# Convert an array of values into a dataset matrix → Sliding Window


def create_dataset(data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data) - look_back):
        a = data[i:(i + look_back+1)]
        dataX.append(a)
        dataY.append(data[i + look_back])
    return np.array(dataX), np.array(dataY)


# Load data
# The each influent data was decomposed into sub-layers, by HHT-EMD

df = pd.read_csv('IMF.csv', delimiter=',')
dataset = df.values
dataset = dataset.T

L, len_IMF =dataset.shape
for iz in range(len_IMF-1):
    IMF=dataset[:,iz]


    # Determine length of train set and test set
    trainL = math.ceil(L * 4 / 5)
    testL = L - trainL

    col = len_IMF - 1

    # Data preparation according to determined length
    look_back = 24  # Window length
    step = +1  # Prediction step (Maximum step must be equal to mxstep)


    # Data Propagation: single IMF !!!

    data = IMF
    trainX = data[:trainL - step, ]
    y_train = data[look_back +step :trainL, ]
    x_train, YY = create_dataset(trainX, look_back)


    testX = data[trainL:L - step]
    y_test = data[look_back + trainL + step:L]
    x_test, YY = create_dataset(testX, look_back)
    # x_test is converted to 3-dimension data by using moving window

    predictX = dataset[L - step - look_back:L]
    x_predict, YY = create_dataset(predictX, look_back)

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)

    # From the sliding window analysis, we divide the Sliding window data into the Sliding window sub-layers
    # Data structure: (Batch size, time step (window size), Sub-layers(HHT-based EMD))

    # Model structure:
    # Input data 1(EMD) ->
    # Encoder: 1D CNN -> RNN(LSTM or GRU) -> Dense
    # Decoder: -> Ensamble -> Forecasting

    x_train= x_train.reshape(1103,25,1)
    x_test = x_test.reshape(256,25,1)

    size_TimeStep, size_Window, size_EMD = x_train.shape

    Input_data = [x_train[:, :, ii].reshape(size_TimeStep, size_Window, 1) for ii in range(size_EMD)]

    model = tf.keras.Sequential()
    model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
    model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
    model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
    model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
    model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
    model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
    model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
    model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
    model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
    model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
    model.add(GRU(32, ))  ## return_sequences=True,))
    # model.add(GRU(16))

    model.add(Dense(16))
    model.add(Dense(4))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy', 'mse', 'mae', 'mape'])

    if iz == len_IMF - 1:
        model = tf.keras.Sequential()

        #model.add(Conv1D(filters=20, kernel_size=7, strides=1, kernel_initializer='random_uniform', padding='same'))
        #model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
        model.add(GRU(32, ))  ## return_sequences=True,))
        # model.add(GRU(16))

        # model.add(Dense(4))
        model.add(Dense(4))
        model.add(Dense(1))
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy', 'mse', 'mae', 'mape'])

    history = model.fit(x_train, y_train, epochs=200, batch_size=128, verbose=0, validation_split=0.1)

    # Generates output predictions for the input samples.
    trainPredict = model.predict(x_train)

    size_TimeStep, size_Window, size_EMD = x_test.shape

    Input_Testdata = [x_test[:, :, ii].reshape(size_TimeStep, size_Window, 1) for ii in range(size_EMD)]

    testPredict = model.predict(x_test)

    # Predict = model.predict(x_predict)

    trainPredictPlot = np.empty_like(data)
    trainPredictPlot[:,] = np.nan
    trainPredictPlot[look_back + step:trainL,] = trainPredict.reshape(len(trainPredict))

    testPredictPlot = np.empty_like(data)
    testPredictPlot[:,] = np.nan
    testPredictPlot[look_back + trainL + step:L,] = testPredict.reshape(len(testPredict))

    # PredictPlot = np.empty([dataset.shape[0]+step,dataset.shape[1]])
    # PredictPlot[:, :] = np.nan
    # PredictPlot[L:L+step, :] = Predict


    # Calculate the forecasting performance
    trainScore = model.evaluate(x_train, y_train, verbose=0)
    testScore = model.evaluate(x_test, y_test, verbose=0)

    test_r2 = r2_score(y_test, testPredict)

    test_mape = mean_absolute_percentage_error(y_test ,testPredict )
    test_mae = mean_absolute_error(y_test, testPredict )
    test_mse = mean_squared_error(y_test , testPredict )

    # Plots the predicted results of the train set and test set


    print('IMF: %.4f |, Test Score | R2: %.4f,  MSE: %.4f, MAE: %.4f,MAPE: %.4f' % (
        iz+1, test_r2,  test_mse, test_mae, test_mape))


    sio.savemat("MEDeepL_result"+str(iz+1)+".mat", {"testPredictPlot": testPredictPlot})


"""
plt.figure()
plt.plot(data[:],'k')
plt.plot(trainPredictPlot,'b')
plt.plot(testPredictPlot,'r')
plt.xlabel('Time')
plt.ylabel('IMF')
plt.show()
"""

GPU is NOT AVAILABLE
(1103, 25)
(1103,)
(256, 25)
(256,)
IMF: 1.0000 |, Test Score | R2: 0.3357,  MSE: 0.0041, MAE: 0.0460,MAPE: 435.7546
(1103, 25)
(1103,)
(256, 25)
(256,)
IMF: 2.0000 |, Test Score | R2: 0.9617,  MSE: 0.0005, MAE: 0.0156,MAPE: 853.0646
(1103, 25)
(1103,)
(256, 25)
(256,)
IMF: 3.0000 |, Test Score | R2: 0.9959,  MSE: 0.0001, MAE: 0.0088,MAPE: 401.1104
(1103, 25)
(1103,)
(256, 25)
(256,)
IMF: 4.0000 |, Test Score | R2: 0.9995,  MSE: 0.0000, MAE: 0.0011,MAPE: 552.9562
(1103, 25)
(1103,)
(256, 25)
(256,)


KeyboardInterrupt: ignored